In [42]:
from pathlib import Path

import cppyy

In [43]:
# include syth library
cppyy.load_library("/home/braden/syth/build/libsyTH_share.so")

# include header file 
cppyy.include("/home/braden/syth/syTH/src/SolidMesh.h")

True

In [44]:
# call package first
cppyy.gbl.syth.SolidMesh

# then we can import
# we can do cppyy.gbl.syth.SolidMesh every time instead of importing
SolidMesh = cppyy.gbl.syth.SolidMesh

# all of the above could probably doesn't have to be done in every file if you get fancy
# I'd see if you could make a python module that does all of this and then import that module
# could do https://cppyy.readthedocs.io/en/latest/python.html#doc-strings

In [45]:
# and now it works!
SolidMesh

<class cppyy.gbl.syth.SolidMesh at 0x56539263fb60>

In [46]:
# emulating test_SolidMesh.cpp
solid_mesh_1 = SolidMesh()
assert (solid_mesh_1.getNCell()==0)
assert (solid_mesh_1.getMaxNeigh()==0)

In [47]:
solid_mesh_2 = SolidMesh(2, 3)
assert solid_mesh_2.getNCell() == 2
assert solid_mesh_2.getMaxNeigh() == 3

nlist = [[n for n in range(i * 3, i * 3 + 3)] for i in range(2)]
solid_mesh_2.setNeighList(nlist)
nlist_out = solid_mesh_2.getNeighList()
for i in range(2):
    for j in range(3):
        assert nlist[i][j] == nlist_out[i][j]

In [48]:
hdf5_example_file_name = "/home/braden/syth/syTH/unit_tests/testSolidMesh/hdf5_reference.h5"
solid_mesh_3 = SolidMesh(hdf5_example_file_name, "mesh/solid_mesh/")

assert solid_mesh_3.getNCell() == 210
assert solid_mesh_3.getMaxNeigh() == 5
nlist = solid_mesh_3.getNeighList()
assert all([nlist[0][i] == value  for i, value in enumerate([-1,1,6,42,-3 ])])

def test_near(x,y,eps=1e-5):
    return abs(x-y)<eps

assert test_near(solid_mesh_3.getFaceArea(0,0), 0.57735)
assert test_near(solid_mesh_3.getFaceArea(0,3), 0.14434)
assert test_near(solid_mesh_3.getFaceArea(1,0), 0.57735)
assert test_near(solid_mesh_3.getFaceArea(1,3), 0.14434)

assert test_near(solid_mesh_3.getVolume(0), 0.14434)
assert test_near(solid_mesh_3.getVolume(1), 0.14434)
assert test_near(solid_mesh_3.getVolume(2), 0.14434)

In [49]:
dp = cppyy.ctypes.c_double()
dx = cppyy.ctypes.c_double()

solid_mesh_3.getDistance2Face(0, 0, dp, dx)
assert test_near(dp.value, 0.16667)
assert test_near(dx.value, 0.0)

solid_mesh_3.getDistance2Face(0, 1, dp, dx)
assert test_near(dp.value, 0.16667)
assert test_near(dx.value, 0.16667)

solid_mesh_3.getDistance2Face(0, 3, dp, dx)
assert test_near(dp.value, 0.5)
assert test_near(dx.value, 0.5)

solid_mesh_3.getDistance2Face(0, 4, dp, dx)
assert test_near(dp.value, 0.5)
assert test_near(dx.value, 0)

In [50]:
# DON'T NEED THIS ANYMORE

# # include external libraries
# cppyy.add_include_path("/home/braden/syth/external/HighFive/include")

# # include syth tpls
# def add_include_paths_recursively(directory):
#     for path in Path(directory).rglob('*'):
#         if path.is_dir():
#             cppyy.add_include_path(str(path))

# # Use the function to add all subdirectories of '~/syth_tpls'
# add_include_paths_recursively('/home/braden/syth_tpls')